In [16]:
import os 
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults 
from langchain.agents import tool
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
from PIL import Image
from IPython.display import display
from pydantic import BaseModel
from typing import List, Optional






In [2]:
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

tavily_tool = TavilySearchResults(api_key =TAVILY_API_KEY)


In [3]:
@tool
def research_agent(input_query: str) -> dict:

    """Use Tavily to get the search results for a query, with a retry on failure."""

    
    
    try:
        search_results = tavily_tool.run(input_query, num_results=10)
    except Exception as e:
        print(f"First attempt failed: {e}. Retrying...")
        try:
            search_results = tavily_tool.run(input_query, num_results=10)
        except Exception as e:
            print(f"Second attempt failed: {e}")
            return {"raw_search_results": "", "original_links": []}

    serialized_results = ""
    original_links = []

    for item in search_results:
        serialized_results += f"Title: {item.get('title')}\nSnippet: {item.get('content')}\nLink: {item.get('url')}\n\n"
        original_links.append(item.get('url'))

    return {"raw_search_results": serialized_results, "original_links": original_links}


In [4]:
# Direct test without graph
query = "What is LangChain?"
research_output = research_agent.invoke({"input_query": query})

print("---- Raw Search Results ----")
print(research_output["raw_search_results"])
print("---- Original Links ----")


---- Raw Search Results ----
Title: What Is LangChain? | DataStax
Snippet: What is LangChain?
LangChain is a Python framework designed to streamline AI application development, focusing on real-time data processing and integration with large language models (LLMs). It offers features for data communication, generating vector embeddings, and simplifying the interaction with LLMs, making it efficient for AI developers. [...] What are the key components of LangChain?
LangChain is a sophisticated framework comprising several key components that work in synergy to enhance natural language processing tasks. These components enable the system to effectively understand, process, and generate human-like language responses.
LLMs (large language models) [...] But LLM interaction is just the beginning of what LangChain can help with. Let’s look at why it is important before diving into some of the framework’s key features.
LangChain is a Python framework designed to streamline AI application devel

In [6]:
@tool
def summarizer_agent(raw_search_results: str, original_links: List[str]) -> dict:
    """Summarizes raw search results into clean bullet points."""
    summarizer_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    prompt = f"""
    You are a research assistant. Summarize the following search results into bullet points.
    Focus only on important facts. Ignore unnecessary details.

    Search Results:
    {raw_search_results}
    """

    response = summarizer_llm.invoke(prompt)
    return {"summary_text": response.content, "original_links": original_links}


In [7]:
# After research_agent passed
summarizer_output = summarizer_agent.invoke(research_output)

print("---- Summary ----")
print(summarizer_output["summary_text"])
print("---- Links Carried ----")
print(summarizer_output["original_links"])


C:\Users\91832\AppData\Local\Temp\ipykernel_19232\7087349.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  summarizer_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


---- Summary ----
- LangChain is a Python framework designed for AI application development, focusing on real-time data processing and integration with large language models (LLMs)
- LangChain streamlines the process of creating generative AI application interfaces and enables developers to easily switch between different LLMs
- LangChain is an open source orchestration framework available in Python and Javascript libraries for building LLM-driven applications like chatbots and virtual agents
- LangChain simplifies the programming of LLM applications through abstraction, representing complex processes as named components
- LangChain facilitates the integration of LLMs into applications for tasks such as document analysis, chatbots, and code analysis
---- Links Carried ----
['https://www.datastax.com/guides/what-is-langchain', 'https://www.techtarget.com/searchenterpriseai/definition/LangChain', 'https://www.ibm.com/think/topics/langchain', 'https://en.wikipedia.org/wiki/LangChain', 'ht

In [ ]:
@tool
def validation_agent(summary_text: str, original_links: List[str]) -> dict:
    """Validates summarized points for correctness."""

    validator_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    prompt = f"""
    You are a fact-checker. Carefully review the following bullet points for accuracy and clarity.
    Correct any mistakes or misleading information.

   {summary_text}
    """

    response = validator_llm.invoke(prompt)
    return {"validated_text": response.content, "original_links": original_links}


In [12]:
# After research_agent passed
validation_output = validation_agent.invoke(summarizer_output)

print("---- Validated summarized points ----")
print(validation_output["validated_text"])
print("---- Links Carried ----")
print(summarizer_output["original_links"])

---- Validated summarized points ----
- LangChain is a Python framework designed for AI application development, focusing on real-time data processing and integration with large language models (LLMs).
- LangChain streamlines the process of creating generative AI application interfaces and enables developers to easily switch between different LLMs.
- LangChain is an open-source orchestration framework available in Python and JavaScript libraries for building LLM-driven applications like chatbots and virtual agents.
- LangChain simplifies the programming of LLM applications through abstraction, representing complex processes as named components.
- LangChain facilitates the integration of LLMs into applications for tasks such as document analysis, chatbots, and code analysis.
---- Links Carried ----
['https://www.datastax.com/guides/what-is-langchain', 'https://www.techtarget.com/searchenterpriseai/definition/LangChain', 'https://www.ibm.com/think/topics/langchain', 'https://en.wikipedia

In [28]:
@tool
def drafting_agent(validated_text: str, original_links: List[str], answer_type: Optional[str] = "detailed") -> str:
    """Drafts the final answer (short or detailed) based on user preference, then appends citations."""
    
    drafting_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

    citations = "\n".join([f"[{i+1}] {link}" for i, link in enumerate(original_links)])

    if answer_type == "short":
        prompt = f"""
        You are an expert research assistant.
        Write a short, concise 1-2 sentence answer based on the following validated content.
        Then list the sources at the end.

        You MUST only use the provided validated content to draft the answer. 
        

        Validated Content:
        {validated_text}

        Sources:
        {citations}
        """
    else:
        prompt = f"""
        You are an expert research answer writer.
        Write a final detailed answer based on the following validated content.
        Then list the sources at the end.
        You MUST only use the provided validated content to draft the answer. 
        

        Validated Content:
        {validated_text}

        Sources:
        {citations}
        """

    response = drafting_llm.invoke(prompt)
    return response.content

In [29]:

final_answer = drafting_agent.invoke({
    "validated_text": validation_output["validated_text"],
    "original_links": validation_output["original_links"],
    "answer_type": "short"  # or"detailed"
})

print(final_answer)


LangChain is a Python framework that simplifies the programming of AI applications by facilitating the integration of large language models (LLMs) for tasks like document analysis and chatbot development. 

Sources:
[1] https://www.datastax.com/guides/what-is-langchain
[2] https://www.techtarget.com/searchenterpriseai/definition/LangChain
[3] https://www.ibm.com/think/topics/langchain
[4] https://en.wikipedia.org/wiki/LangChain
[5] https://python.langchain.com/docs/introduction/


In [30]:
final_answer = drafting_agent.invoke({
    "validated_text": validation_output["validated_text"],
    "original_links": validation_output["original_links"],
    "answer_type": "detailed"  
})

print(final_answer)

LangChain is a cutting-edge Python framework that has been specifically developed for AI application development, with a primary focus on real-time data processing and seamless integration with large language models (LLMs). This framework serves as a pivotal tool in the creation of generative AI application interfaces, allowing developers to effortlessly switch between various LLMs. 

One of the key aspects of LangChain is its open-source nature, which enables developers to leverage its capabilities through Python and JavaScript libraries. By utilizing LangChain, developers can build LLM-driven applications such as chatbots and virtual agents with ease. The framework simplifies the programming of LLM applications by abstracting complex processes into named components, thereby enhancing the overall development process.

LangChain plays a crucial role in facilitating the integration of LLMs into a wide range of applications, including document analysis, chatbots, and code analysis. This 

In [25]:
# Defining the Langgraph work Flow
graph = StateGraph(input="input_query", output="final_answer")


# adding the two agents workflow 
graph.add_node("research", research_agent)
graph.add_node("summarize",summarizer_agent)
graph.add_node("validate", validation_agent)
graph.add_node("draft",drafting_agent)

# seting  up the flow 

graph.set_entry_point("research")

graph.add_edge("research","summarize")
graph.add_edge("summarize","validate")
graph.add_edge("validate","draft")

graph.set_finish_point("draft")


# compile the graph 

graph_app = graph.compile()



C:\Users\91832\AppData\Local\Temp\ipykernel_19232\631017967.py:2: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  graph = StateGraph(input="input_query", output="final_answer")
c:\Users\91832\Desktop\Tavily_web\tavily\lib\site-packages\langgraph\graph\state.py:89: UserWarning: Invalid state_schema: input_query. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(
c:\Users\91832\Desktop\Tavily_web\tavily\lib\site-packages\langgraph\graph\state.py:89: UserWarning: Invalid state_schema: final_answer. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(


In [31]:
query = "what is langchain and its importance?"
answer_type = "short"

results = graph_app.invoke({"input_query": query})


#print the final results

print("Final Answer :")

print(results)

Final Answer :
LangChain is an open-source framework designed for developing applications utilizing large language models (LLMs). It focuses on streamlining AI application development, specifically emphasizing real-time data processing and integration with LLMs. By leveraging LangChain, industries such as customer service, content creation, and data analysis can experience enhancements in efficiency, accuracy, and contextual relevance of AI interactions. 

One of the key advantages of LangChain is its ability to bridge the communication gap between humans and machines, thereby making AI more accessible and practical in various applications. Developers can utilize LangChain to construct applications that combine LLMs with external sources of computation and data, ultimately simplifying the creation process and providing a standard interface for chains and integrations with other tools. 

Moreover, LangChain tools enable agents to interact with real-world information to expand or enhance